# 2 Basic Flight Control - Flying to the Car

## Large Model Function Encapsulation  
We encapsulate the io.net LLM API into a function suitable for drone control, named `airsim_agent.py`.

Only one main function is used:

In [ ]:
def process(self, command,run_python_code=True):
    #step 1, ask llm
    response = self.ask(command)

    #step 2, extract Python code
    python_code = self.extract_python_code(response)

    #step 3, run Python code
    if run_python_code and python_code:
        exec(python_code)
    return python_code

run method：

In [2]:
import airsim_agent
my_agent = airsim_agent.AirSimAgent()
command = "take off"
python_code = my_agent.process(command) #not run code
print("python_code:", python_code)

python_code: aw.takeoff()



In [ ]:
# agent ini funciton：
# def __init__(self, system_prompts="system_prompts/airsim_basic_cn.txt", knowledge_prompt="prompts/airsim_basic_cn.txt", chat_history=[]):
    

In [3]:
#write knowladge into aisim_lession23.txt
kg_promot_file = "prompts/aisim_lession23.txt"

kg_prompt = """
Below are some functions you can use to command the drone.

`aw.takeoff()` - Takes off the drone.  
`aw.land()` - Lands the drone.

Please respond in the following format:  
```python
i=1  # Here, output the Python code
```
This code function is for assignment.

You do not need to worry about importing `aw`, as it has already been declared in the environment.
"""

pt_file = open(kg_promot_file, "w", encoding="utf-8")
pt_file.write(kg_prompt)
pt_file.close()

In [4]:
#Attention，if you need run the drone code，you need open airsim first
import airsim_agent
my_agent = airsim_agent.AirSimAgent(knowledge_prompt="prompts/aisim_lession23.txt")
command = "take off"
python_code = my_agent.process(command, True) #run the code
print("python_code:", python_code)

python_code: aw.takeoff()#起飞无人机



## Flying to the Car

Since the drone currently lacks visual capabilities, we can directly specify the target position coordinates.

You can obtain the object's pose using the following function:  
`client.simGetObjectPose(object_name)`

Because the object names used in the API may differ from their actual names in the Unreal Engine environment, we can use a dictionary to map them:

```python
objects_dict = {
    "turbine1": "BP_Wind_Turbines_C_1",
    "turbine2": "StaticMeshActor_2",
    "solarpanels": "StaticMeshActor_146",
    "crowd": "StaticMeshActor_6",
    "car": "StaticMeshActor_10",
    "tower1": "SM_Electric_trellis_179",
    "tower2": "SM_Electric_trellis_7",
    "tower3": "SM_Electric_trellis_8",
}
```

Furthermore, if you are using a non-English language (e.g., Chinese), since variable names in code are typically not written in Chinese, you only need to provide the mapping between the Chinese object name and its corresponding English identifier within the prompt.

#build new promption，write into aisim_lession24.txt
kg_promot_file = "prompts/aisim_lession24.txt"

kg_prompt = """
Below are some functions you can use to command the drone:

- `aw.takeoff()` - Takes off the drone.  
- `aw.land()` - Lands the drone.  
- `aw.get_drone_position()` - Returns the current position of the drone as a list of three floating-point numbers corresponding to the X, Y, and Z coordinates.  
- `aw.fly_to([x, y, z])` - Flies the drone to the specified position, given as a list of three parameters corresponding to the X, Y, and Z coordinates.  
- `aw.get_position(object_name)` - Accepts a string input indicating the name of the object of interest, and returns a list of three floating-point numbers representing its X, Y, and Z coordinates.

The following objects exist in the scene, and you should refer to them using **exactly** these names:

**WindTurbine1, WindTurbine2, SolarPanels, Car, Crowd, Tower1, Tower2, Tower3**

Please note that when calling functions, if you're using a non-English language such as Chinese, you should still use the English identifiers listed below. The mapping is as follows:

- `turbine1`: 风力发电机1  
- `turbine2`: 风力发电机2  
- `solarpanels`: 太阳能电池板  
- `car`: 汽车  
- `crowd`: 人群  
- `tower1`: 塔1  
- `tower2`: 塔2  
- `tower3`: 塔3  

For example, to get the position of WindTurbine1, you would write:  
```python
aw.get_position("turbine1")
```  
instead of:  
```python
aw.get_position("风力发电机1")
```

All objects other than the drone itself are stationary. Also, please remember that there are two turbines and three towers. When multiple objects of the same type exist, **you should always ask for clarification** if it’s not explicitly stated which one is being referred to. **Never make assumptions.**

Please respond in the following format:  
```python
i=1  # Here, output the Python code
```
This code function is for assignment.

You do not need to worry about importing `aw`, as it has already been declared in the environment.

"""

pt_file = open(kg_promot_file, "w", encoding="utf-8")
pt_file.write(kg_prompt)
pt_file.close()

In [7]:
import airsim_agent
my_agent = airsim_agent.AirSimAgent(knowledge_prompt="prompts/aisim_lession24.txt")
command = "take off"
python_code = my_agent.process(command, True) #run the code
print("python_code: \n", python_code)

python_code: 
 aw.takeoff()



In [8]:
command = "Fly to above the car"
python_code = my_agent.process(command, False) #not run
print("python_code: \n", python_code)

python_code: 
 car_position = aw.get_position("car")
# 假设飞到汽车上方 5 米处，根据 NED 坐标系，向上飞 z 轴减去一个值
target_position = [car_position[0], car_position[1], car_position[2] - 5]
aw.fly_to(target_position)



In [9]:
command = "Fly to above the car"
python_code = my_agent.process(command, True) #run the code
print("python_code: \n", python_code)

python_code: 
 # 获取汽车的位置
car_pos = aw.get_position("car")
# 假设飞到汽车上方 3 米处，由于是 NED 坐标系，向上飞 z 轴值减小
target = [car_pos[0], car_pos[1], car_pos[2] - 3]
# 让无人机飞到目标位置
aw.fly_to(target)

